In [1]:
import numpy as np
import matplotlib.pyplot as plt
import SiPMWaveGen as swg
import scipy.signal as scisig

In [2]:
def FindPedestal(p, m):
    noOutlier = RejectOutliers(p, m=m)
    return np.mean(noOutlier)

In [3]:
def FindDigitizedPedestal(p, m, nBits, dynamicRange):
    noOutlier = FindPedestal(p=p, m=m)
    res = dynamicRange / (2 ** nBits - 1)
    noiseInADC = swg.getRawADC(speAmplitude / 15, res)

    return [np.mean(noOutlier), noiseInADC]


In [4]:
def RejectOutliers(data, m=2.):
    d = np.abs(data - np.median(data))
    mdev = np.median(d)
    s = d/mdev if mdev else 0.
    return data[s<m]

In [9]:
def WaveformDiscriminator(p,
                          nNoiseSigmaThreshold=1,
                          sgFilter=True,
                          sgWindow=15,
                          sgPolyOrder=3):
    [baselineVal, noiseInADC] = FindDigitizedPedestal(p=p, m=3, nBits=12, dynamicRange=1)
    if sgFilter:
        filter_p = scisig.savgol_filter(x=p, window_length=sgWindow, polyorder=sgPolyOrder)
        hitLogic = np.array([(True if pi < baselineVal - nNoiseSigmaThreshold * noiseInADC else False) for pi in filter_p])
    else:
        hitLogic = np.array([(True if pi < baselineVal - nNoiseSigmaThreshold * noiseInADC else False) for pi in p])
    return [hitLogic, baselineVal, noiseInADC]


In [10]:
def DiscriminatorConditioning(p,
                              durationTheshold=5,
                              adjDurationThreshold=5,
                              nNoiseSigmaThreshold=1,
                              sgFilter=True,
                              sgWindow=15,
                              sgPolyOrder=3):
    [hitLogic, baseline, noiseSigma] = WaveformDiscriminator(p=p, 
                                     nNoiseSigmaThreshold=nNoiseSigmaThreshold, 
                                     sgFilter=sgFilter, 
                                     sgWindow=sgWindow, 
                                     sgPolyOrder=sgPolyOrder)
    
    for i in range(1, np.size(hitLogic)):
        if ((not hitLogic[i-1]) and hitLogic[i]) and hitLogic[i]:
            countDuration = 0
            for j in range(i, np.size(hitLogic)-1):
                if hitLogic[j]:
                    countDuration = countDuration + 1
                if not hitLogic[j+1]:
                    break
                    
            if countDuration < durationTheshold:
                for j in range(i, i + countDuration):
                    hitLogic[j] = False
                    
    for i in range(1, np.size(hitLogic)):
        if (hitLogic[i-1] and (not hitLogic[i])) and (not hitLogic[i]):
            countDuration = 0
            for j in range(i, np.size(hitLogic)-1):
                if (not hitLogic[j]):
                    countDuration = countDuration + 1
                if hitLogic[j+1]:
                    break
                    
            if countDuration < adjDurationThreshold:
                for j in range(i, i + countDuration):
                    hitLogic[j] = True
    
    return [hitLogic, baseline, noiseSigma]
        

In [40]:
def HitFinder(p,
              cfdThreshold=0.2,
              durationTheshold=5,
              adjDurationThreshold=5,
              nNoiseSigmaThreshold=1,
              sgFilter=True,
              sgWindow=15,
              sgPolyOrder=3):
    [hitLogic, baseline, noiseSigma] = DiscriminatorConditioning(p=p,
                         durationTheshold=durationTheshold,
                         adjDurationThreshold=adjDurationThreshold,
                         nNoiseSigmaThreshold=nNoiseSigmaThreshold,
                         sgFilter=sgFilter,
                         sgWindow=sgWindow,
                         sgPolyOrder=sgPolyOrder)
    
    hitStartIndexList = np.zeros(0)
    for i in range(1, np.size(hitLogic)):
        if ((not hitLogic[i-1]) and hitLogic[i]) and hitLogic[i]:
            hitAmplitude = 1E100
            hitPeakIndex = i
            for j in range(i, np.size(hitLogic)-1):
                if p[j] < hitAmplitude:
                    hitAmplitude = p[j]
                    hitPeakIndex = j
                if not hitLogic[j+1]:
                    break
            ThresholdADC = baseline - (cfdThreshold * (baseline - hitAmplitude))
            
            hitStartIndex = i
            exactStartIndex = False
            for j in range(hitPeakIndex, 0, -1):
                if p[j] == ThresholdADC:
                    hitStartIndex = j
                    exactStartIndex = True
                    break
                if (p[j] < ThresholdADC and p[j - 1] > ThresholdADC):
                    hitStartIndex = j
                    break
                    
            if exactStartIndex:
                hitStartIndexList = np.append(hitStartIndexList, hitStartIndex)
            else:
                if hitStartIndex >= 3:
                    V2 = p[hitStartIndex]
                    t2 = hitStartIndex
                    V1 = p[hitStartIndex - 1]
                    t1 = hitStartIndex - 1
                    m = (V2 - V1) / (t2 - t1)
                    extrapolatedHitStartIndex = ((ThresholdADC - V1) / m) + t1
                    hitStartIndexList = np.append(hitStartIndexList, extrapolatedHitStartIndex)
    
    return [hitStartIndexList, hitLogic, baseline, noiseSigma]

In [48]:
timeSample = 0.2  # nanosec
nSamples = 1024  # no. of sampling points in a trigger
speAmplitude = 0.15  # single photon-electron response amplitude in volt, noise sigma is 1/10 of the speAmplitude
riseTime = 1.5  # nanosec
fallTime = 3  # nanosec
nBits = 12  # 12bit ADC
voltMin = -0.9  # minimum voltage ADC in volt
dynamicRange = 1  # dynamic range of ADC
offset = 2000  # offset ADC

cfdThreshold = 0.2  # CFD threshold
durationTheshold = 10  # hit finder alg param: duration of a detected peak to be considered a hit
adjDurationThreshold = 10  # hit finder alg param: duration between two detected peaks to be considered a single hit
nNoiseSigmaThreshold = 1.5  # adcThreshold = (noise threshold) * nNoiseSigmaThreshold + baseline
sgFilter = True  # using Golay-Savitzky filter 
sgWindow = 15  # sg sliding window
sgPolyOrder = 3  # sg smoothing polynomial order

for i in range(10):
    [t, p, true_p] = swg.aDigitizedTrigger(dt=timeSample,
                                           nsamples=nSamples,
                                           speAmplitude=speAmplitude,
                                           riseTime=riseTime,
                                           fallTime=fallTime,
                                           nBits=nBits,
                                           voltMin=voltMin,
                                           dynamicRange=dynamicRange,
                                           offset=offset)

    [hitList, hitLogic, baseline, noiseSigma] = HitFinder(p=p,
                                                          cfdThreshold=cfdThreshold,
                                                          durationTheshold=durationTheshold,
                                                          adjDurationThreshold=adjDurationThreshold,
                                                          nNoiseSigmaThreshold=nNoiseSigmaThreshold,
                                                          sgFilter=sgFilter,
                                                          sgWindow=sgWindow,
                                                          sgPolyOrder=sgPolyOrder)
    
    filter_p = scisig.savgol_filter(x=p, 
                                    window_length=sgWindow, 
                                    polyorder=sgPolyOrder)
    if np.size(hitList) > 0:
        plt.plot(t, p, )
        for x in hitList:
            if x < 1024 and x >= 0:
                plt.axvline(x=(x*0.2))
        plt.show()
    
